In [1]:
import pandas as pd
import numpy as np
import torch
import joblib
from datetime import datetime

In [2]:
# 모델 및 스케일러 로드
def load_model(model_path, scaler_X_path, scaler_y_path):
    # 장치 설정
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    
    print(f"Using device: {device}")
    
    # 모델 로드
    checkpoint = torch.load(model_path, map_location=device)
    
    # 모델 구성 정보 가져오기
    config = checkpoint['model_config']
    
    # LSTMModel 클래스 재정의 (원래 코드와 동일해야 함)
    from torch import nn
    
    class LSTMModel(nn.Module):
        def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob=0.2):
            super(LSTMModel, self).__init__()
            
            self.hidden_size = hidden_size
            self.num_layers = num_layers
            
            self.lstm = nn.LSTM(
                input_size=input_size,
                hidden_size=hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout_prob if num_layers > 1 else 0
            )
            
            self.dropout = nn.Dropout(dropout_prob)
            self.fc = nn.Linear(hidden_size, output_size)
            
        def forward(self, x):
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
            
            out, _ = self.lstm(x, (h0, c0))
            out = out[:, -1, :]
            out = self.dropout(out)
            out = self.fc(out)
            
            return out
    
    # 모델 인스턴스 생성
    model = LSTMModel(
        input_size=config['input_size'],
        hidden_size=config['hidden_size'],
        num_layers=config['num_layers'],
        output_size=config['output_size'],
        dropout_prob=config['dropout_prob']
    ).to(device)
    
    # 모델 가중치 로드
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # 스케일러 로드
    scaler_X = joblib.load(scaler_X_path)  # 실제 경로로 변경
    scaler_y = joblib.load(scaler_y_path)  # 실제 경로로 변경
    
    return model, scaler_X, scaler_y, device

In [3]:
# 테스트 데이터 읽기 및 처리
def process_test_data(test_file_path, seq_length=24):
    # CSV 파일 읽기
    test_df = pd.read_csv(test_file_path)
    
    # prediction_time 열을 datetime 형식으로 변환
    test_df['prediction_time'] = pd.to_datetime(test_df['prediction_time'], dayfirst=True)
    
    # 시간 특성 추가
    test_df['hour'] = test_df['prediction_time'].dt.hour
    test_df['day_of_week'] = test_df['prediction_time'].dt.dayofweek
    test_df['month'] = test_df['prediction_time'].dt.month
    test_df['day'] = test_df['prediction_time'].dt.day
    
    return test_df

In [4]:
# 시간 데이터로 예측 수행
def predict_power_consumption(model, test_df, scaler_X, scaler_y, device, seq_length=24):
    model.eval()  # 평가 모드 설정
    
    # 시간 특성 추출
    time_features = test_df[['hour', 'day_of_week', 'month', 'day']].values
    
    # 전체 입력 특성 준비 (나머지는 0으로 채움)
    # 모든 특성을 0으로 초기화
    full_features = np.zeros((len(test_df), scaler_X.n_features_in_))
    
    # 시간 특성 채우기 - 실제 특성 순서에 맞게 조정 필요
    # 예: 시간 특성이 마지막 4개 열이라고 가정
    feature_indices = list(range(scaler_X.n_features_in_ - 4, scaler_X.n_features_in_))
    full_features[:, feature_indices] = time_features
    
    # 입력 정규화
    normalized_features = scaler_X.transform(full_features)
    
    with torch.no_grad():
        # 입력을 텐서로 변환
        # LSTM은 [batch_size, seq_length, input_size] 형태의 입력을 기대
        # 각 시간점에 대해 독립적으로 예측하므로 seq_length=1로 설정
        input_tensor = torch.tensor(normalized_features, dtype=torch.float32).unsqueeze(1).to(device)
        
        # 예측
        predictions = model(input_tensor).cpu().numpy()
        
        # 예측값 역정규화
        predictions_original = scaler_y.inverse_transform(predictions)
    
    return predictions_original.flatten()

In [5]:
# 결과를 원하는 형식으로 저장
def save_predictions(test_df, predictions, output_file='predicted_power_loads.csv'):
    # 예측값 반올림 (정수로)
    
    # 시간과 예측값 사이에 쉼표 구분자 추가하여 CSV 생성
    rows = []
    for i, dt in enumerate(test_df['prediction_time']):
        # 날짜/시간 형식 설정 (앞의 0 제거)
        day = dt.day
        month = dt.month
        year = dt.year
        hour = dt.hour
        minute = dt.minute
        
        # 분이 0인 경우 00으로 표시
        if minute == 0:
            minute_str = "00"
        else:
            minute_str = str(minute)
        
        # 시간 형식 생성
        time_str = f"{day}/{month}/{year} {hour}:{minute_str}"
        
        # 행 추가 (시간,예측값)
        rows.append(f"{time_str},{predictions[i]}")
    
    # 행들을 줄바꿈으로 구분하여 파일에 저장 (헤더는 포함하지 않음)
    with open(output_file, 'w') as f:
        f.write("\n".join(rows))
    
    print(f"예측 결과가 {output_file}에 저장되었습니다.")
    
    return rows

In [7]:

test_file_path = 'data/raw-data/test.csv'  # 테스트 데이터 파일 경로
output_file = 'predicted_power_loads.csv'  # 결과 저장 파일 경로
model, scaler_X, scaler_y, device = load_model("result/first/chiller_lstm_model_first.pth", "result/first/scaler_X_first.pkl", "result/first/scaler_y_first.pkl")

# 테스트 데이터 처리
test_df = process_test_data(test_file_path)

# 예측 수행
predictions = predict_power_consumption(model, test_df, scaler_X, scaler_y, device)

# 결과 저장
save_predictions(test_df, predictions, output_file)

Using device: mps
예측 결과가 predicted_power_loads.csv에 저장되었습니다.


/Users/jsyoon/Library/Caches/pypoetry/virtualenvs/pure-lstm-mtv0ePvh-py3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


['1/1/2024 0:00,24.395200729370117',
 '1/1/2024 1:00,22.77091407775879',
 '1/1/2024 2:00,21.246923446655273',
 '1/1/2024 3:00,19.819046020507812',
 '1/1/2024 4:00,18.482074737548828',
 '1/1/2024 5:00,17.2301082611084',
 '1/1/2024 6:00,16.05682945251465',
 '1/1/2024 7:00,14.955763816833496',
 '1/1/2024 8:00,13.920458793640137',
 '1/1/2024 9:00,12.944656372070312',
 '1/1/2024 10:00,12.022378921508789',
 '1/1/2024 11:00,11.148024559020996',
 '1/1/2024 12:00,10.316386222839355',
 '1/1/2024 13:00,9.522697448730469',
 '1/1/2024 14:00,8.762619972229004',
 '1/1/2024 15:00,8.032235145568848',
 '1/1/2024 16:00,7.328022480010986',
 '1/1/2024 17:00,6.646857738494873',
 '1/1/2024 18:00,5.985954284667969',
 '1/1/2024 19:00,5.342851638793945',
 '1/1/2024 20:00,4.715394496917725',
 '1/1/2024 21:00,4.101694583892822',
 '1/1/2024 22:00,3.5001072883605957',
 '1/1/2024 23:00,2.9092061519622803',
 '2/1/2024 0:00,25.72223663330078',
 '2/1/2024 1:00,24.17089080810547',
 '2/1/2024 2:00,22.709339141845703',
 '